In [1]:
# 노트북 최상단에 추가
import os
import sys

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from language_backend.constant import DEFAULT_CHUNK_SIZE, DEFAULT_CHUNK_OVERLAP

In [3]:
from langchain_teddynote.korean import stopwords
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob


In [4]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=DEFAULT_CHUNK_SIZE, 
    chunk_overlap=DEFAULT_CHUNK_OVERLAP
)

In [5]:


split_docs = []

# 텍스트 파일을 load -> List[Document] 형태로 변환
files = sorted(glob.glob("../datasets/research_report/20241021/industry/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

In [6]:
split_docs

[Document(metadata={'source': '../datasets/research_report/20241021/industry/20241021_industry_39280000.pdf', 'file_path': '../datasets/research_report/20241021/industry/20241021_industry_39280000.pdf', 'page': 0, 'total_pages': 9, 'format': 'PDF 1.5', 'title': '', 'author': '박혜숙', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20241018164917+09'00'", 'modDate': "D:20241018164917+09'00'", 'trapped': ''}, page_content='2024년 10월 21일 \n기업분석 \n \n \n \n더본코리아 (상장예정) \n물 들어올 때 노 젓는다지만 \n \n▶Analyst 한유정 yujung.han@hanwha.com 3772-7693 / RA 이다연dayeon.lee@hanwha.com 3772-7690 \n더본코리아는? \n더본코리아는 1994년 설립된 외식 브랜드 운영 기업으로 백종원 대표가 설립한 회사다. 2023년 매출 기준 외식사업 매\n출이 85.9%로 가장 높은 비중을 차지하고 있다. 외식사업은 프랜차이즈 가맹 사업으로 주요 비즈니스 모델은 가맹본부\n와 가맹점 간의 가맹 계약에 기반한 사업 모델이다. 2023년 기준 외식 사업 매출액 3,528억원 중 88%가 가맹점포에 식\n자재를 유통, 판매해 판매대금을 수취하는 상품 및 제품 매출에서 발생되었으며 5%는 가맹비 및 로열티에서 발생되었다.  \n더본코리아의 경쟁력: 1) 브랜드 확장, 2) 해외 진출'),
 Document(metadata={'source

In [38]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv('../.dev.env')

True

In [12]:
from language_crawler.database.models import ArticleContentOrm

import pandas as pd
from sqlalchemy import create_engine

# 데이터베이스에서 데이터 로드
engine = create_engine('sqlite:///../datasets/naver_news.db')
query = "SELECT * FROM article_contents"
df = pd.read_sql(query, engine)

In [20]:
df

,id,article_id,media_id,content,title,language,article_published_at,article_modified_at,created_at,html,ticker
0,1,0005300993,009,\n피에스케이홀딩스는 피에스케이 주식회사가 2019년 인적 분할하면서 설립된 회사다...,매니저의 HOT종목,ko,2024-05-09 16:05:08.000000,None,2024-05-17 14:26:36.387884,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,211270
1,2,0000285776,629,\n-이번에는 금융권 소식을 들어보겠습니다. 미국 중앙은행인 연방준비제도(Fed)가...,[비즈토크<하>] 美금리 6연속 동결에…셈법 복잡해진 한은,ko,2024-05-05 00:04:03.000000,None,2024-05-17 14:26:36.471548,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,6840
2,3,0004326341,011,\n\t\t\t[서울경제] <유가증권>▲보통주추가상장=(주식매수선택권행사) (국내C...,오늘의 증시 메모[4월 12일],ko,2024-04-11 14:17:03.000000,None,2024-05-17 14:26:36.488026,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,109960
3,4,0007502685,421,\n(서울=뉴스1) 박현영 기자 = 한국 진출을 예고했던 해외 대형 가상자산(암호화...,바이낸스도 크립토닷컴도 '진입금지'…해외 거래소는 못 오는 한국,ko,2024-04-25 07:00:00.000000,None,2024-05-17 14:26:36.518673,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,139050
4,5,0005733933,018,\n\t\t\t[이데일리 이정현 기자]AP시스템(265520)은 연결재무제표 기준 ...,"AP시스템, 1Q 영업익 72.3억…전년비 38.17%↓",ko,2024-05-08 16:38:02.000000,None,2024-05-17 14:26:36.546228,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,265520
...,...,...,...,...,...,...,...,...,...,...,...
765,766,0000813319,031,"\nKT&G 차기 사장 후보군이 내부 출신 2명, 외부 2명 등 4명으로 압축됐다....",KT&G 사장 '숏리스트'…내부 방경만·허철호 vs 외부 권계현·이석주,ko,2024-02-16 18:31:01.000000,None,2024-05-17 14:27:09.323844,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,6840
766,767,0005380903,277,\nKT&G는 16일 열린 사장후보추천위원회(사추위)에서 차기 사장 후보 2차 숏리...,"KT&G, 사장 후보 4명 압축…권계현·방경만·이석주·허철호",ko,2024-02-16 18:23:50.000000,None,2024-05-17 14:27:09.354115,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,6840
767,768,0000071858,050,\nKT&G 사장후보추천위원회(이하 사추위)는 16일 사추위를 열고 차기 사장 후보...,KT&G 사장 후보 4명 압축…권계현·방경만·이석주·허철호,ko,2024-02-16 18:52:01.000000,None,2024-05-17 14:27:09.383846,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,6840
768,769,0003817143,023,\n\t\t\t\t\t\t\t\t\t\tKT&G 사장후보추천위원회(사추위)는 권계현...,KT&G 사장 후보 4명으로 압축…내·외부 인사 각 2명,ko,2024-02-16 18:23:01.000000,None,2024-05-17 14:27:09.413646,b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x...,6840


In [13]:
from language_backend.constant import NAVER_NEWS_ARTICLE_COLLECTION 

In [14]:

from language_backend.vectorstore import get_client


client = get_client()
if client.has_collection(NAVER_NEWS_ARTICLE_COLLECTION):
    client.drop_collection(NAVER_NEWS_ARTICLE_COLLECTION)


In [15]:
from langchain_openai import OpenAIEmbeddings


embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [30]:
%reload_ext autoreload
%autoreload 2

In [34]:
from language_backend.vectorstore import get_naver_news_article_collection

In [40]:
collection = get_naver_news_article_collection()

UsageError: Line magic function `%export` not found.


In [41]:
for row in df.itertuples():
    collection.insert([
        {
            "article_id": int(row.article_id),            
            "ticker": row.ticker,
            "title": row.title,
            "content": row.content,
            "content_embedding": embedding.embed_query(row.content),
            "article_published_at": row.article_published_at,
            "article_modifed_at": row.article_modifed_at,
        }
    ])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-vHz2R***************************************6Cbu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}